<a href="https://colab.research.google.com/github/Shallom12/Autonomous-driving-of-Chungnam-Human-Resources-Development-Institute/blob/main/peopleNet%2CtrafficNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# 필요한 라이브러리 설치
# yt-dlp: YouTube 비디오 다운로드 (pytube 대체)
# ultralytics: YOLOv5 모델 사용
# opencv-python: 비디오 처리
# torch: 딥러닝 프레임워크
!pip install yt-dlp ultralytics opencv-python torch

import cv2
import numpy as np
import yt_dlp
import torch
from ultralytics import YOLO
import os
from google.colab import files

# YouTube 비디오 다운로드 함수
def download_youtube_video(url, output_path):
    """
    주어진 YouTube URL에서 비디오를 다운로드하여 지정된 경로에 저장합니다.
    yt-dlp를 사용하여 안정적으로 다운로드합니다.

    Args:
        url (str): YouTube 비디오 URL
        output_path (str): 다운로드된 비디오를 저장할 경로
    Returns:
        str: 다운로드된 비디오 파일의 경로
    """
    try:
        # yt-dlp 옵션 설정
        ydl_opts = {
            'format': 'best[ext=mp4]',  # MP4 형식의 최고 품질 비디오 선택
            'outtmpl': os.path.join(output_path, 'input_video.mp4'),  # 출력 파일명
            'quiet': False,  # 진행 상황 출력
            'no_warnings': True,  # 경고 메시지 억제
        }

        print(f"비디오 다운로드 중: {url}")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            print(f"다운로드 완료: {info['title']}")
        return os.path.join(output_path, 'input_video.mp4')
    except Exception as e:
        print(f"비디오 다운로드 오류: {str(e)}")
        return None

# 비디오 처리 및 사람 감지 함수
def process_video(video_path, output_path, model):
    """
    비디오를 처리하여 사람을 감지하고, 결과를 저장합니다.

    Args:
        video_path (str): 입력 비디오 경로
        output_path (str): 출력 비디오 저장 경로
        model: 사람 감지를 위한 사전 학습된 모델 (YOLOv5)
    """
    # 비디오 열기
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("오류: 비디오를 열 수 없습니다.")
        return

    # 비디오 속성 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 출력 비디오 설정 (코덱 및 파일 생성)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 프레임을 RGB로 변환 (YOLOv5는 RGB 입력을 요구)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 사람 감지 수행
        results = model(frame_rgb)

        # 감지 결과 처리
        # YOLOv5 결과 형식 변경에 대한 처리
        # ultralytics 라이브러리의 YOLOv5 결과는 xyxy[0] 대신 results[0].boxes.data를 사용합니다.
        if results and results[0].boxes:
            for detection in results[0].boxes.data:  # [x1, y1, x2, y2, 신뢰도, 클래스]
                if int(detection[5]) == 0:  # YOLOv5에서 클래스 0은 'person'
                    x1, y1, x2, y2 = map(int, detection[:4])
                    confidence = detection[4]
                    if confidence > 0.5:  # 신뢰도 임계값
                        # 바운딩 박스와 라벨 그리기
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(frame, f'사람 {confidence:.2f}', (x1, y1-10),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


        # 처리된 프레임을 출력 비디오에 저장
        out.write(frame)

    # 자원 해제
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"처리된 비디오가 {output_path}에 저장되었습니다.")

# 메인 실행 함수
def main():
    # 경로 설정
    video_url = "https://youtu.be/KoRmB706ZTA?si=Et_huXMkr3L7d_tT"
    output_dir = "/content"
    output_video_path = "/content/output_video.mp4"

    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)

    # YouTube 비디오 다운로드
    video_path = download_youtube_video(video_url, output_dir)

    # 비디오 다운로드 성공 여부 확인 후 처리 진행
    if video_path and os.path.exists(video_path):
        # YOLOv5 모델 로드 (Colab에서 빠른 추론을 위해 'yolov5s' 사용)
        print("YOLOv5 모델 로드 중...")
        model = YOLO('yolov5s.pt')  # 사전 학습된 YOLOv5 소형 모델

        # 비디오 처리
        print("사람 감지를 위해 비디오 처리 중...")
        process_video(video_path, output_video_path, model)

        # 처리된 비디오 다운로드 (파일이 존재하는 경우에만 시도)
        if os.path.exists(output_video_path):
            print("처리된 비디오 다운로드 중...")
            files.download(output_video_path)
        else:
            print("오류: 처리된 비디오 파일을 찾을 수 없어 다운로드할 수 없습니다.")
    else:
        print("YouTube 비디오 다운로드에 실패하여 처리를 진행할 수 없습니다.")


if __name__ == "__main__":
    main()

비디오 다운로드 중: https://youtu.be/KoRmB706ZTA?si=Et_huXMkr3L7d_tT
[youtube] Extracting URL: https://youtu.be/KoRmB706ZTA?si=Et_huXMkr3L7d_tT
[youtube] KoRmB706ZTA: Downloading webpage
[youtube] KoRmB706ZTA: Downloading tv client config
[youtube] KoRmB706ZTA: Downloading player 0b00c3eb-main
[youtube] KoRmB706ZTA: Downloading tv player API JSON
[youtube] KoRmB706ZTA: Downloading ios player API JSON
[youtube] KoRmB706ZTA: Downloading m3u8 information
[info] KoRmB706ZTA: Downloading 1 format(s): 18
[download] /content/input_video.mp4 has already been downloaded
[download] 100% of    3.49MiB
다운로드 완료: “지하철 9호선, 출근 시간대 운행 횟수 늘릴 것” / KBS  2023.07.31.
YOLOv5 모델 로드 중...
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

사람 감지를 위해 비디오 처리 중...

0: 384x640 8 persons, 2 ties, 1 laptop, 1 mouse, 165.2m

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# 필요한 라이브러리 설치
# yt-dlp: YouTube 비디오 다운로드
# ultralytics: YOLOv5 모델 사용
# opencv-python: 비디오 처리
# torch: 딥러닝 프레임워크
# pillow: 한글 텍스트 렌더링
!pip install yt-dlp ultralytics opencv-python torch pillow

import cv2
import numpy as np
import yt_dlp
import torch
from ultralytics import YOLO
import os
from google.colab import files
from PIL import Image, ImageDraw, ImageFont

# YouTube 비디오 다운로드 함수
def download_youtube_video(url, output_path):
    """
    주어진 YouTube URL에서 비디오를 다운로드하여 지정된 경로에 저장합니다.
    yt-dlp를 사용하여 안정적으로 다운로드합니다.

    Args:
        url (str): YouTube 비디오 URL
        output_path (str): 다운로드된 비디오를 저장할 경로
    Returns:
        str: 다운로드된 비디오 파일의 경로
    """
    try:
        # yt-dlp 옵션 설정
        ydl_opts = {
            'format': 'best[ext=mp4]',  # MP4 형식의 최고 품질 비디오 선택
            'outtmpl': os.path.join(output_path, 'input_video.mp4'),  # 출력 파일명
            'quiet': False,  # 진행 상황 출력
            'no_warnings': True,  # 경고 메시지 억제
        }

        print(f"비디오 다운로드 중: {url}")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            print(f"다운로드 완료: {info['title']}")
        return os.path.join(output_path, 'input_video.mp4')
    except Exception as e:
        print(f"비디오 다운로드 오류: {str(e)}")
        return None

# 한글 텍스트를 프레임에 추가하는 함수
def draw_text_pil(frame, text, position, font_path="/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc", font_size=20):
    """
    PIL을 사용하여 OpenCV 프레임에 한글 텍스트를 그립니다.

    Args:
        frame: OpenCV 프레임 (numpy 배열)
        text: 그릴 텍스트 (예: "사람 0.95")
        position: 텍스트 위치 (x, y)
        font_path: 한글 폰트 파일 경로
        font_size: 폰트 크기
    Returns:
        frame: 텍스트가 추가된 프레임
    """
    # OpenCV 프레임을 PIL 이미지로 변환
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(frame_pil)

    # 폰트 로드
    try:
        font = ImageFont.truetype(font_path, font_size)
    except Exception as e:
        print(f"폰트 로드 오류: {e}. 기본 폰트 사용")
        font = ImageFont.load_default()

    # 텍스트 그리기
    draw.text(position, text, font=font, fill=(0, 255, 0, 255))  # 초록색 텍스트

    # PIL 이미지를 OpenCV 프레임으로 변환
    frame = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)
    return frame

# 비디오 처리 및 사람 감지 함수
def process_video(video_path, output_path, model):
    """
    비디오를 처리하여 사람을 감지하고, 결과를 저장합니다.

    Args:
        video_path (str): 입력 비디오 경로
        output_path (str): 출력 비디오 저장 경로
        model: 사람 감지를 위한 사전 학습된 모델 (YOLOv5)
    """
    # 비디오 열기
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("오류: 비디오를 열 수 없습니다.")
        return

    # 비디오 속성 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 출력 비디오 설정 (코덱 및 파일 생성)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 프레임을 RGB로 변환 (YOLOv5는 RGB 입력을 요구)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 사람 감지 수행
        results = model(frame_rgb)

        # 감지 결과 처리
        if results and results[0].boxes:
            for detection in results[0].boxes.data:  # [x1, y1, x2, y2, 신뢰도, 클래스]
                class_id = int(detection[5])
                if class_id == 0:  # YOLOv5에서 클래스 0은 'person'
                    x1, y1, x2, y2 = map(int, detection[:4])
                    confidence = float(detection[4])
                    if confidence > 0.5:  # 신뢰도 임계값
                        # 바운딩 박스 그리기
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        # 한글 라벨 추가 (PIL 사용)
                        label = f"사람 {confidence:.2f}"
                        frame = draw_text_pil(frame, label, (x1, y1-10))
                        # 디버깅용 출력
                        print(f"감지: 클래스={model.names[class_id]}, 신뢰도={confidence:.2f}, 좌표=({x1}, {y1}, {x2}, {y2})")

        # 처리된 프레임을 출력 비디오에 저장
        out.write(frame)

    # 자원 해제
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"처리된 비디오가 {output_path}에 저장되었습니다.")

# 메인 실행 함수
def main():
    # 경로 설정
    video_url = "https://youtu.be/KoRmB706ZTA?si=Et_huXMkr3L7d_tT"
    output_dir = "/content"
    output_video_path = "/content/output_video.mp4"

    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)

    # YouTube 비디오 다운로드
    video_path = download_youtube_video(video_url, output_dir)

    # 비디오 다운로드 성공 여부 확인 후 처리 진행
    if video_path and os.path.exists(video_path):
        # YOLOv5 모델 로드 (Colab에서 빠른 추론을 위해 'yolov5s' 사용)
        print("YOLOv5 모델 로드 중...")
        model = YOLO('yolov5s.pt')  # 사전 학습된 YOLOv5 소형 모델

        # 클래스 이름 확인 (디버깅용)
        print(f"모델 클래스 이름: {model.names}")

        # 비디오 처리
        print("사람 감지를 위해 비디오 처리 중...")
        process_video(video_path, output_video_path, model)

        # 처리된 비디오 다운로드 (파일이 존재하는 경우에만 시도)
        if os.path.exists(output_video_path):
            print("처리된 비디오 다운로드 중...")
            files.download(output_video_path)
        else:
            print("오류: 처리된 비디오 파일을 찾을 수 없어 다운로드할 수 없습니다.")
    else:
        print("YouTube 비디오 다운로드에 실패하여 처리를 진행할 수 없습니다.")

if __name__ == "__main__":
    main()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
감지: 클래스=person, 신뢰도=0.59, 좌표=(130, 167, 266, 284)
폰트 로드 오류: cannot open resource. 기본 폰트 사용
감지: 클래스=person, 신뢰도=0.52, 좌표=(552, 248, 613, 329)

0: 384x640 13 persons, 164.4ms
Speed: 1.4ms preprocess, 164.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
폰트 로드 오류: cannot open resource. 기본 폰트 사용
감지: 클래스=person, 신뢰도=0.89, 좌표=(342, 123, 429, 282)
폰트 로드 오류: cannot open resource. 기본 폰트 사용
감지: 클래스=person, 신뢰도=0.83, 좌표=(362, 104, 566, 358)
폰트 로드 오류: cannot open resource. 기본 폰트 사용
감지: 클래스=person, 신뢰도=0.74, 좌표=(246, 155, 329, 283)
폰트 로드 오류: cannot open resource. 기본 폰트 사용
감지: 클래스=person, 신뢰도=0.70, 좌표=(4, 120, 131, 360)
폰트 로드 오류: cannot open resource. 기본 폰트 사용
감지: 클래스=person, 신뢰도=0.63, 좌표=(231, 104, 280, 234)
폰트 로드 오류: cannot open resource. 기본 폰트 사용
감지: 클래스=person, 신뢰도=0.56, 좌표=(299, 117, 349, 281)
폰트 로드 오류: cannot open resource. 기본 폰트 사용
감지: 클래스=person, 신뢰도=0.52, 좌표=(187, 126, 247, 246)
폰트 로드 오류: cannot open resource. 기본 폰트 사용
감지: 클래스=person, 신

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 필요한 라이브러리 설치
# yt-dlp: YouTube 비디오 다운로드
# ultralytics: YOLOv5 모델 사용
# opencv-python: 비디오 처리
# torch: 딥러닝 프레임워크
!pip install yt-dlp ultralytics opencv-python torch

import cv2
import numpy as np
import yt_dlp
import torch
from ultralytics import YOLO
import os
from google.colab import files

# YouTube 비디오 다운로드 함수
def download_youtube_video(url, output_path):
    """
    주어진 YouTube URL에서 비디오를 다운로드하여 지정된 경로에 저장합니다.
    yt-dlp를 사용하여 안정적으로 다운로드합니다.

    Args:
        url (str): YouTube 비디오 URL
        output_path (str): 다운로드된 비디오를 저장할 경로
    Returns:
        str: 다운로드된 비디오 파일의 경로
    """
    try:
        # yt-dlp 옵션 설정
        ydl_opts = {
            'format': 'best[ext=mp4]',  # MP4 형식의 최고 품질 비디오 선택
            'outtmpl': os.path.join(output_path, 'input_video.mp4'),  # 출력 파일명
            'quiet': False,  # 진행 상황 출력
            'no_warnings': True,  # 경고 메시지 억제
        }

        print(f"비디오 다운로드 중: {url}")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            print(f"다운로드 완료: {info['title']}")
        return os.path.join(output_path, 'input_video.mp4')
    except Exception as e:
        print(f"비디오 다운로드 오류: {str(e)}")
        return None

# 비디오 처리 및 사람 감지 함수
def process_video(video_path, output_path, model):
    """
    비디오를 처리하여 사람을 감지하고, 결과를 저장합니다.

    Args:
        video_path (str): 입력 비디오 경로
        output_path (str): 출력 비디오 저장 경로
        model: 사람 감지를 위한 사전 학습된 모델 (YOLOv5)
    """
    # 비디오 열기
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("오류: 비디오를 열 수 없습니다.")
        return

    # 비디오 속성 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 출력 비디오 설정 (코덱 및 파일 생성)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 프레임을 RGB로 변환 (YOLOv5는 RGB 입력을 요구)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 사람 감지 수행
        results = model(frame_rgb)

        # 감지 결과 처리
        if results and results[0].boxes:
            for detection in results[0].boxes.data:  # [x1, y1, x2, y2, 신뢰도, 클래스]
                class_id = int(detection[5])
                if class_id == 0:  # YOLOv5에서 클래스 0은 'person'
                    x1, y1, x2, y2 = map(int, detection[:4])
                    confidence = float(detection[4])
                    if confidence > 0.5:  # 신뢰도 임계값
                        # 바운딩 박스 그리기
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        # 영어 라벨 추가
                        label = f"Person {confidence:.2f}"
                        cv2.putText(frame, label, (x1, y1-10),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        # 디버깅용 출력
                        print(f"감지: 클래스={model.names[class_id]}, 신뢰도={confidence:.2f}, 좌표=({x1}, {y1}, {x2}, {y2})")

        # 처리된 프레임을 출력 비디오에 저장
        out.write(frame)

    # 자원 해제
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"처리된 비디오가 {output_path}에 저장되었습니다.")

# 메인 실행 함수
def main():
    # 경로 설정
    video_url = "https://youtu.be/uLC9ZpIavhI?si=FZijyhzvEeavjDt_"
    output_dir = "/content"
    output_video_path = "/content/output_video.mp4"

    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)

    # YouTube 비디오 다운로드
    video_path = download_youtube_video(video_url, output_dir)

    # 비디오 다운로드 성공 여부 확인 후 처리 진행
    if video_path and os.path.exists(video_path):
        # YOLOv5 모델 로드 (Colab에서 빠른 추론을 위해 'yolov5s' 사용)
        print("YOLOv5 모델 로드 중...")
        model = YOLO('yolov5s.pt')  # 사전 학습된 YOLOv5 소형 모델

        # 클래스 이름 확인 (디버깅용)
        print(f"모델 클래스 이름: {model.names}")

        # 비디오 처리
        print("사람 감지를 위해 비디오 처리 중...")
        process_video(video_path, output_video_path, model)

        # 처리된 비디오 다운로드 (파일이 존재하는 경우에만 시도)
        if os.path.exists(output_video_path):
            print("처리된 비디오 다운로드 중...")
            files.download(output_video_path)
        else:
            print("오류: 처리된 비디오 파일을 찾을 수 없어 다운로드할 수 없습니다.")
    else:
        print("YouTube 비디오 다운로드에 실패하여 처리를 진행할 수 없습니다.")

if __name__ == "__main__":
    main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 17.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
비디오 다운로드 중: https://youtu.be/uLC9ZpIavhI?si=FZijyhzvEeavjDt_
[youtube] Extracting URL: https://youtu.be/uLC9ZpIavhI?si=FZijyhzvEeavjDt_
[youtube] uLC9ZpIavhI: Downloading webpage
[youtube] uLC9ZpIavhI: Downloading tv client config
[youtube] uLC9ZpIavhI: Downloading player 662eb823-main
[youtube] uLC9ZpIavhI: Downloading tv player API JSON
[youtube] uLC9ZpIavhI: Downloading

100%|██████████| 17.7M/17.7M [00:00<00:00, 140MB/s] 


모델 클래스 이름: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 

In [1]:
# 필요한 라이브러리 설치
# yt-dlp: YouTube 비디오 다운로드
# ultralytics: YOLOv5 모델 사용
# opencv-python: 비디오 처리
# torch: 딥러닝 프레임워크
!pip install yt-dlp ultralytics opencv-python torch

import cv2
import numpy as np
import yt_dlp
import torch
from ultralytics import YOLO
import os
from google.colab import files

# YouTube 비디오 다운로드 함수
def download_youtube_video(url, output_path):
    """
    주어진 YouTube URL에서 비디오를 다운로드하여 지정된 경로에 저장합니다.
    yt-dlp를 사용하여 안정적으로 다운로드합니다.

    Args:
        url (str): YouTube 비디오 URL
        output_path (str): 다운로드된 비디오를 저장할 경로
    Returns:
        str: 다운로드된 비디오 파일의 경로
    """
    try:
        ydl_opts = {
            'format': 'best[ext=mp4]',  # MP4 형식의 최고 품질 비디오 선택
            'outtmpl': os.path.join(output_path, 'input_video.mp4'),  # 출력 파일명
            'quiet': False,  # 진행 상황 출력
            'no_warnings': True,  # 경고 메시지 억제
        }

        print(f"비디오 다운로드 중: {url}")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            print(f"다운로드 완료: {info['title']}")
        return os.path.join(output_path, 'input_video.mp4')
    except Exception as e:
        print(f"비디오 다운로드 오류: {str(e)}")
        return None

# 비디오 처리 및 차량 감지 함수
def process_video(video_path, output_path, model):
    """
    비디오를 처리하여 차량을 감지하고, 결과를 저장합니다.
    YOLOv5를 사용하여 슈퍼카(예: car, truck, bus 등)를 감지.

    Args:
        video_path (str): 입력 비디오 경로
        output_path (str): 출력 비디오 저장 경로
        model: 차량 감지를 위한 사전 학습된 모델
    """
    # 비디오 열기
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("오류: 비디오를 열 수 없습니다.")
        return

    # 비디오 속성 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 출력 비디오 설정 (코덱 및 파일 생성)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 2 == 0:  # 프레임 속도 줄이기 (성능 최적화)
            continue

        # 프레임을 RGB로 변환 (YOLOv5는 RGB 입력을 요구)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 차량 감지 수행
        results = model(frame_rgb, conf=0.3)  # 신뢰도 임계값 0.3으로 설정

        # 감지 결과 처리
        if results and results[0].boxes:
            for detection in results[0].boxes.data:  # [x1, y1, x2, y2, 신뢰도, 클래스]
                class_id = int(detection[5])
                # YOLOv5 COCO 데이터셋에서 차량 관련 클래스
                vehicle_classes = [2, 5, 7]  # car: 2, bus: 5, truck: 7
                if class_id in vehicle_classes:
                    x1, y1, x2, y2 = map(int, detection[:4])
                    confidence = float(detection[4])
                    if confidence > 0.3:  # 낮춘 신뢰도 임계값 적용
                        # 바운딩 박스 그리기
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        # 영어 라벨 추가
                        class_names = ["car", "bus", "truck"]  # vehicle_classes에 맞춘 순서
                        label = f"{class_names[vehicle_classes.index(class_id)]} {confidence:.2f}"
                        cv2.putText(frame, label, (x1, y1-10),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        print(f"프레임 {frame_count}: 감지 - 클래스={class_names[vehicle_classes.index(class_id)]}, 신뢰도={confidence:.2f}, 좌표=({x1}, {y1}, {x2}, {y2})")

        # 처리된 프레임을 출력 비디오에 저장
        out.write(frame)

    # 자원 해제
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"처리된 비디오가 {output_path}에 저장되었습니다.")

# 메인 실행 함수
def main():
    # 경로 설정
    video_url = "https://youtu.be/b_P7qRD2czM?si=4VhT26mQHrLpC8Ik"
    output_dir = "/content"
    output_video_path = "/content/output_video.mp4"

    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)

    # YouTube 비디오 다운로드
    video_path = download_youtube_video(video_url, output_dir)

    # 비디오 다운로드 성공 여부 확인 후 처리 진행
    if video_path and os.path.exists(video_path):
        # YOLOv5 모델 로드 (더 강력한 'yolov5m' 사용)
        print("YOLOv5 모델 로드 중...")
        model = YOLO('yolov5m.pt')  # yolv5m.pt로 업그레이드

        # 클래스 이름 확인 (디버깅용)
        print(f"모델 클래스 이름: {model.names}")

        # 비디오 처리
        print("차량 감지를 위해 비디오 처리 중...")
        process_video(video_path, output_video_path, model)

        # 처리된 비디오 다운로드 (파일이 존재하는 경우에만 시도)
        if os.path.exists(output_video_path):
            print("처리된 비디오 다운로드 중...")
            files.download(output_video_path)
        else:
            print("오류: 처리된 비디오 파일을 찾을 수 없어 다운로드할 수 없습니다.")
    else:
        print("YouTube 비디오 다운로드에 실패하여 처리를 진행할 수 없습니다.")

if __name__ == "__main__":
    main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 17.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
비디오 다운로드 중: https://youtu.be/b_P7qRD2czM?si=4VhT26mQHrLpC8Ik
[youtube] Extracting URL: https://youtu.be/b_P7qRD2czM?si=4VhT26mQHrLpC8Ik
[youtube] b_P7qRD2czM: Downloading webpage
[youtube] b_P7qRD2czM: Downloading tv client config
[youtube] b_P7qRD2czM: Downloading player 662eb823-main
[youtube] b_P7qRD2czM: Downloading tv player API JSON
[youtube] b_P7qRD2czM: Downloading

100%|██████████| 48.2M/48.2M [00:00<00:00, 138MB/s]


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
프레임 3733: 감지 - 클래스=car, 신뢰도=0.43, 좌표=(1, 1, 635, 359)

0: 384x640 1 car, 271.4ms
Speed: 1.1ms preprocess, 271.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
프레임 3735: 감지 - 클래스=car, 신뢰도=0.46, 좌표=(1, 1, 635, 359)

0: 384x640 1 car, 218.1ms
Speed: 1.2ms preprocess, 218.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
프레임 3737: 감지 - 클래스=car, 신뢰도=0.38, 좌표=(1, 11, 631, 356)

0: 384x640 (no detections), 265.1ms
Speed: 1.2ms preprocess, 265.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 211.7ms
Speed: 1.1ms preprocess, 211.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 268.9ms
Speed: 1.1ms preprocess, 268.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 219.6ms
Speed: 1.3ms preprocess, 219.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>